In [1]:
import numpy as np
import librosa
import os

def pre_emphasis(signal_in, pre_emph=0.97):
    """
    Áp dụng pre-emphasis để nhấn mạnh các tần số cao.
    """
    return np.append(signal_in[0], signal_in[1:] - pre_emph * signal_in[:-1])

def framing(signal_in, sample_rate, frame_size=0.025, frame_stride=0.01):
    """
    Chia tín hiệu thành các frame có kích thước và bước nhảy xác định.
    """
    frame_length = int(round(frame_size * sample_rate))
    frame_step = int(round(frame_stride * sample_rate))
    signal_length = len(signal_in)
    num_frames = int(np.ceil(np.abs(signal_length - frame_length) / frame_step)) + 1

    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(signal_in, z)

    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + \
              np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    return frames

def windowing(frames):
    """
    Áp dụng cửa sổ Hamming cho mỗi frame.
    """
    frame_length = frames.shape[1]
    hamming = np.hamming(frame_length)
    return frames * hamming

def fft_frames(frames, NFFT=512, high_pass_cutoff=50, low_pass_cutoff=4000):
    """
    Tính FFT cho mỗi frame và lấy giá trị magnitude, sau đó lọc các tần số cao.
    - high_pass_cutoff: ngưỡng tần số cắt cao (Hz).
    - low_pass_cutoff: ngưỡng tần số cắt thấp (Hz).
    """
    # Tính FFT
    mag_frames = np.absolute(np.fft.rfft(frames, NFFT))
    
    # Lọc các tần số cao và thấp
    freq_bins = np.fft.fftfreq(NFFT, 1.0 / frames.shape[1])
    freq_bins = freq_bins[:NFFT // 2 + 1]  # Chỉ lấy nửa phần thực của FFT
    
    # Áp dụng bộ lọc high-pass và low-pass
    high_pass_mask = freq_bins >= high_pass_cutoff
    low_pass_mask = freq_bins <= low_pass_cutoff
    filter_mask = high_pass_mask & low_pass_mask

    # Áp dụng bộ lọc cho magnitude
    mag_frames = mag_frames * filter_mask
    
    return mag_frames

def compute_fft_features(signal_in, sample_rate, frame_size=0.025, frame_stride=0.01, NFFT=512, apply_log=True, high_pass_cutoff=50, low_pass_cutoff=4000):
    """
    Tính toán đặc trưng FFT cho tín hiệu âm thanh:
      - Pre-emphasis, Framing, Windowing.
      - Tính FFT cho từng frame và lấy giá trị magnitude.
      - Trung bình các frame để có vector đặc trưng ổn định.
      - (Tùy chọn) Áp dụng log để giảm phạm vi giá trị.
      - Áp dụng bộ lọc high-pass và low-pass để giảm nhiễu.
    """
    emphasized_signal = pre_emphasis(signal_in)
    frames = framing(emphasized_signal, sample_rate, frame_size, frame_stride)
    windowed_frames = windowing(frames)
    mag_frames = fft_frames(windowed_frames, NFFT, high_pass_cutoff, low_pass_cutoff)
    fft_feature = np.mean(mag_frames, axis=0)  # Trung bình theo các frame
    if apply_log:
        fft_feature = np.log(fft_feature + 1e-8)  # Thêm epsilon để tránh log(0)
    return fft_feature

def load_fft_features_from_directory(directory, sample_rate=22050, NFFT=512, high_pass_cutoff=50, low_pass_cutoff=4000):
    """
    Duyệt qua các file âm thanh trong thư mục và tính đặc trưng FFT cho mỗi file.
    Giả sử trong directory có hai thư mục con: 'Queen' và 'NonQueen'.
    
    Trả về:
      - features: mảng đặc trưng (mỗi đặc trưng có kích thước NFFT/2+1)
      - labels: nhãn tương ứng.
    """
    labels = []
    features = []
    for label in ['Queen', 'NonQueen']:
        path = os.path.join(directory, label)
        for file in os.listdir(path):
            file_path = os.path.join(path, file)
            signal, sr = librosa.load(file_path, sr=sample_rate)
            fft_feature = compute_fft_features(signal, sr, NFFT=NFFT, high_pass_cutoff=high_pass_cutoff, low_pass_cutoff=low_pass_cutoff)
            features.append(fft_feature)
            labels.append(label)
    return np.array(features), np.array(labels)

# Cập nhật thư mục đường dẫn
train_dir = r'E:\Queenless\20k_audio_splitted_dataset\train'
val_dir   = r'E:\Queenless\20k_audio_splitted_dataset\val'
test_dir  = r'E:\Queenless\20k_audio_splitted_dataset\test'

# Tính toán đặc trưng FFT cho các tập dữ liệu với ngưỡng lọc tần số
train_features, train_labels = load_fft_features_from_directory(train_dir, sample_rate=22050, NFFT=512, high_pass_cutoff=50, low_pass_cutoff=4000)
val_features, val_labels = load_fft_features_from_directory(val_dir, sample_rate=22050, NFFT=512, high_pass_cutoff=50, low_pass_cutoff=4000)
test_features, test_labels = load_fft_features_from_directory(test_dir, sample_rate=22050, NFFT=512, high_pass_cutoff=50, low_pass_cutoff=4000)


In [2]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

In [3]:
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC


# Khởi tạo mô hình SVM với data scaling
svm_rbf_classifier = SVC(C=100, kernel='rbf', degree=3, gamma=0.1, coef0=0.0, shrinking=True, 
                         probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                         max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=42)

# Huấn luyện mô hình SVM
svm_rbf_classifier.fit(train_features_scaled, train_labels)

# Đánh giá mô hình trên bộ validation
val_predictions_svm_rbf = svm_rbf_classifier.predict(val_features_scaled)
val_accuracy_svm_rbf = accuracy_score(val_labels, val_predictions_svm_rbf)
print(f"Validation Accuracy (SVM with RBF Kernel): {val_accuracy_svm_rbf * 100:.2f}%")

# Đánh giá mô hình trên bộ testing
test_predictions_svm_rbf = svm_rbf_classifier.predict(test_features_scaled)
scale_test_accuracy_fft_svm = accuracy_score(test_labels, test_predictions_svm_rbf)
print(f"Test Accuracy (SVM with RBF Kernel): {scale_test_accuracy_fft_svm * 100:.2f}%")

Validation Accuracy (SVM with RBF Kernel): 93.03%
Test Accuracy (SVM with RBF Kernel): 92.70%


In [4]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import numpy as np

# Chuẩn bị dữ liệu (bạn có thể thêm bước chuẩn hóa dữ liệu tại đây)
scaler = StandardScaler()
train_features_scaled = scaler.fit_transform(train_features)
val_features_scaled = scaler.transform(val_features)
test_features_scaled = scaler.transform(test_features)

# Mô hình SVM với kernel RBF
svm_rbf_classifier = SVC(C=100, kernel='rbf', degree=3, gamma=0.1, coef0=0.0, shrinking=True, 
                         probability=False, tol=0.001, cache_size=200, class_weight=None, verbose=False, 
                         max_iter=-1, decision_function_shape='ovr', break_ties=False, random_state=42)

# Định nghĩa các giá trị cần thử cho high-pass và low-pass cutoff
high_pass_cutoffs = [20, 50, 100, 150, 200]  # Ngưỡng tần số cắt high-pass (Hz)
low_pass_cutoffs = [1000, 2000, 3000, 4000]  # Ngưỡng tần số cắt low-pass (Hz)

# Tạo một danh sách các tham số cần thử
param_grid = {
    'high_pass_cutoff': high_pass_cutoffs,
    'low_pass_cutoff': low_pass_cutoffs
}

# Hàm để tính toán đặc trưng FFT với bộ lọc high-pass và low-pass
def compute_filtered_features(features, high_pass_cutoff, low_pass_cutoff):
    return [compute_fft_features(sig, 22050, NFFT=512, high_pass_cutoff=high_pass_cutoff, low_pass_cutoff=low_pass_cutoff) for sig in features]

# Hàm tìm kiếm ngưỡng tối ưu
def grid_search_optimized_thresholds(train_features, train_labels, val_features, val_labels, param_grid):
    best_accuracy = 0
    best_hp_cutoff = None
    best_lp_cutoff = None

    for high_pass_cutoff in param_grid['high_pass_cutoff']:
        for low_pass_cutoff in param_grid['low_pass_cutoff']:
            # Tính toán đặc trưng đã lọc với các ngưỡng khác nhau
            train_features_filtered = compute_filtered_features(train_features, high_pass_cutoff, low_pass_cutoff)
            val_features_filtered = compute_filtered_features(val_features, high_pass_cutoff, low_pass_cutoff)

            # Huấn luyện mô hình SVM
            svm_rbf_classifier.fit(train_features_filtered, train_labels)

            # Dự đoán trên bộ validation
            val_predictions = svm_rbf_classifier.predict(val_features_filtered)
            accuracy = accuracy_score(val_labels, val_predictions)

            # Cập nhật ngưỡng tốt nhất
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_hp_cutoff = high_pass_cutoff
                best_lp_cutoff = low_pass_cutoff

    return best_hp_cutoff, best_lp_cutoff, best_accuracy

# Chạy Grid Search để tìm ngưỡng tối ưu
best_hp_cutoff, best_lp_cutoff, best_accuracy = grid_search_optimized_thresholds(
    train_features_scaled, train_labels, val_features_scaled, val_labels, param_grid)

print(f"Ngưỡng high-pass tốt nhất: {best_hp_cutoff} Hz")
print(f"Ngưỡng low-pass tốt nhất: {best_lp_cutoff} Hz")
print(f"Độ chính xác tốt nhất trên bộ validation: {best_accuracy * 100:.2f}%")

# Tính toán độ chính xác trên bộ kiểm tra (test set)
test_features_filtered = compute_filtered_features(test_features_scaled, best_hp_cutoff, best_lp_cutoff)
test_predictions = svm_rbf_classifier.predict(test_features_filtered)
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Độ chính xác trên bộ kiểm tra: {test_accuracy * 100:.2f}%")


Ngưỡng high-pass tốt nhất: 100 Hz
Ngưỡng low-pass tốt nhất: 1000 Hz
Độ chính xác tốt nhất trên bộ validation: 64.68%
Độ chính xác trên bộ kiểm tra: 50.00%
